General concepts of working with Orbit, Ephem, and Phys objects
===============================================================

The classes Orbit, Ephem, and Phys are all derived from the same base class ([DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass)), meaning that they can all be used in the same way. 

The core of [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) is an astropy [QTable](http://docs.astropy.org/en/stable/api/astropy.table.QTable.html#astropy.table.QTable) object, which is an astropy [Table](http://docs.astropy.org/en/stable/table/) that is aware of [units](http://docs.astropy.org/en/stable/units/). [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) provides a number of convenience functions that make it easy to create and modify the underlying [QTable](http://docs.astropy.org/en/stable/api/astropy.table.QTable.html#astropy.table.QTable) object. If these convenience functions do not cover what you are trying to do, you can directly address and modify the [QTable](http://docs.astropy.org/en/stable/api/astropy.table.QTable.html#astropy.table.QTable) object.

For some introductional examples on how to create, access, and modify [DataClass](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass) objects, please have a look at the [documentation](https://sbpy.readthedocs.io/en/latest/sbpy/data.html).

In the following sections, we provide some examples that are close to real use cases.

Adding a column and converting units
---------------------------

We obtain ephemerides for our target asteroid from JPL Horizons.

In [1]:
from astropy.time import Time
from sbpy.data import Ephem

epoch1 = Time('2018-09-01 12:00', scale='utc')
epoch2 = Time('2018-09-30 12:00', scale='utc')
eph = Ephem.from_horizons('2018 RE3',
                          location='568',
                          epochs={'start': epoch1,
                                  'stop': epoch2,
                                  'step': '5d'},
                          skip_daylight=True)
print(eph.table)

targetname    datetime_str   datetime_jd   H    ... alpha_true  PABLon   PABLat 
                                  d       mag   ...    deg       deg      deg   
---------- ----------------- ----------- ------ ... ---------- -------- --------
(2018 RE3) 2018-Sep-01 12:00   2458363.0 27.149 ...     9.2118  338.904   5.2608
(2018 RE3) 2018-Sep-06 12:00   2458368.0 27.149 ...    13.1477 340.6554   6.4865
(2018 RE3) 2018-Sep-11 12:00   2458373.0 27.149 ...    30.8255 340.0625  13.4257
(2018 RE3) 2018-Sep-16 12:00   2458378.0 27.149 ...   170.1979  76.7881  43.6054
(2018 RE3) 2018-Sep-21 12:00   2458383.0 27.149 ...   163.5381  80.6809  -7.0835
(2018 RE3) 2018-Sep-26 12:00   2458388.0 27.149 ...   156.7853  83.0719 -10.5409


Sky motion rates are provided for both RA and Dec in units of arcsec per hour:

In [2]:
print(eph['RA_rate', 'DEC_rate'])

 RA_rate    DEC_rate 
arcsec / h arcsec / h
---------- ----------
  -62.6186   36.74156
  -144.784    97.0164
  -1184.16   952.1415
  -430.016   -484.145
   -39.151   -72.7347
  0.061824   -28.8482


Let's assume that we are interested in the absolute sky motion in units of arcsec per second. It would be handy to add a column to `eph` that contains this quantity. This can be easily done using the [add_column](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass.add_column) function. Finally, we want to print a table containing epoch, the distance to the observer, and the corresponding absolute sky motion:

In [3]:
import numpy as np
eph.add_column(np.sqrt(eph['RA_rate']**2 + eph['DEC_rate']**2).to('arcsec/second'), name='abs_rate')
print(eph['datetime_str', 'delta', 'abs_rate'])

   datetime_str        delta             abs_rate      
                         AU             arcsec / s     
----------------- ---------------- --------------------
2018-Sep-01 12:00 0.07211862315233 0.020167185670536096
2018-Sep-06 12:00 0.04195739080355   0.0484119639178449
2018-Sep-11 12:00 0.01287432734201  0.42207676054461973
2018-Sep-16 12:00 0.01822288335488  0.17987267043099792
2018-Sep-21 12:00 0.04763409103743 0.022945078994941484
2018-Sep-26 12:00 0.07801247284629 0.008013407290781224


The function [add_column](https://sbpy.readthedocs.io/en/latest/api/sbpy.data.DataClass.html#sbpy.data.DataClass.add_column) takes two parameters: the data in form of a list or an array, and the name of the new column. The data that we provide here consists of two parts. We derive the absolute sky motion as the geometric sum of the RA and Dec rates (`np.sqrt(eph['RA_rate']**2 + eph['DEC_rate']**2)`). Remember that each of these rates is in units of arcsec per hour. In order to convert the rates to arcsec per sec, we have to apply `.to('arcsec/second')` to this array. Finally, we assign a name to this new column.

Filtering table content
-----------------------

We have a list of 5 asteroids that we would like to observe on the night of 2018-09-12 at the Discovery Channel telescope. However, we don't want to observe these asteroids when they have an absolute sky motion rate faster than 0.1 arcsec/s and when the Moon is up.

In [4]:
import numpy as np
import astropy.units as u
from astropy.time import Time
from sbpy.data import Ephem

# target list
targets = ['2018 RR4', '2018 RE3', '2018 RC4', '2018 RQ2', '2018 RC1']

epoch1 = Time('2018-09-13 00:00', scale='utc')
epoch2 = Time('2018-09-14 00:00', scale='utc')
eph = Ephem.from_horizons(targets,
                          location='G37',
                          epochs={'start': epoch1,
                                  'stop': epoch2,
                                  'step': '10m'},
                          skip_daylight=True)
print(len(eph.table))

345


A total of 345 ephemerides have been queried for the 5 asteroids. Now we apply the absolute sky motion rate (see example above) and moon filters; if the moon is up, `eph['flags']` will be set to `'m'` (see https://ssd.jpl.nasa.gov/?horizons_doc&table_quantities#table_quantities):

In [5]:
eph = eph[np.sqrt(eph['RA_rate']**2 + eph['DEC_rate']**2).to('arcsec/second') < 0.1*u.arcsec/u.second]
eph = eph[eph['flags'] != 'm'] 

print(eph)

targetname    datetime_str      datetime_jd    ... alpha_true  PABLon  PABLat
                                     d         ...    deg       deg     deg  
---------- ----------------- ----------------- ... ---------- -------- ------
(2018 RC1) 2018-Sep-13 04:00 2458374.666666667 ...     8.2231 350.8126 4.4013
(2018 RC1) 2018-Sep-13 04:10 2458374.673611111 ...     8.2183 350.8203 4.3985
(2018 RC1) 2018-Sep-13 04:20 2458374.680555556 ...     8.2135  350.828 4.3957
(2018 RC1) 2018-Sep-13 04:30      2458374.6875 ...     8.2087 350.8357 4.3929
(2018 RC1) 2018-Sep-13 04:40 2458374.694444444 ...     8.2039 350.8434 4.3901
(2018 RC1) 2018-Sep-13 04:50 2458374.701388889 ...     8.1992  350.851 4.3873
(2018 RC1) 2018-Sep-13 05:00 2458374.708333333 ...     8.1944 350.8586 4.3845
(2018 RC1) 2018-Sep-13 05:10 2458374.715277778 ...     8.1896 350.8661 4.3817
(2018 RC1) 2018-Sep-13 05:20 2458374.722222222 ...     8.1849 350.8737 4.3789
(2018 RC1) 2018-Sep-13 05:30 2458374.729166667 ...     8.1801 35

As it turns out, only asteroid 2018 RC1 is observable under these conditions.